# Using AWS Lambda and PyWren for Web Scraping and Sentiment Analysis

This notebook is a demonstration of how to perform a web scrape, determine frequent words and build tag clouds massively parallel with AWS Lambda and PyWren. We will be using the [GDELT](https://aws.amazon.com/public-datasets/gdelt/) public dataset to determine which news articles to scrape, we then use NLTK to stem and tokenize words, perform a sentiment analysis and count frequent words. Lastly we will create tag clouds of frequent words and make them available in an Amazon S3 bucket.

### Credits
- [NLTK](http://www.nltk.org/) - NLTK is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning.
- [PyWren](https://github.com/pywren/pywren) - Project by BCCI and riselab. Makes it easy to executive massive parallel map queries across [AWS Lambda](https://aws.amazon.com/lambda/)
- [Wordcloud](https://github.com/amueller/word_cloud) - A little word cloud generator in Python created by [Andres Mueller](https://github.com/amueller)
- [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) - Beautiful Soup is a Python library designed for quick turnaround projects like screen-scraping

# Step by Step instructions

### Setup Logging (optional)
Only activate the below lines if you want to see all debug messages from PyWren. _Note: The output will be rather chatty and lengthy._

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
%env PYWREN_LOGLEVEL=INFO

Let's setup all the necessary libraries:

In [ ]:
import boto3, botocore
from IPython.display import HTML, display, Image, IFrame
import pywren
import GDELT_scrape

wrenexec = pywren.default_executor()

def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

**IMPORTANT** - We need to update the S3 Bucket variable with the Amazon S3 bucket that has been created with the AWS Cloudformation template earlier. Please update the following variable with your bucketname that you copied out of the Output tab.

<img src="../Images/cloudformation-s3.png" style="width: 700px;"/>

In [ ]:
S3BUCKET = 'pywren-workshop-s3bucket-12apl9us09h1d'

Now let's use PyWren to collect a number of GDELT listed articles by polling the GDELT data from the GDELT AWS Dataset from Amazon S3. Whilst PyWren is processing you can watch the Cloudwatch Logs for output from the function - navigate to the [Cloudwatch Logs Console](https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/lambda/pywren_1;start=PT5M).

If you want to understand the exact details, explore [GDELT_scrape.py](/edit/Lab-3-Scrape-Sentiment/GDELT_scrape.py). Here are the relevant code snippets. We basically open up the requested CSV file in Amazon S3 in the AWS Lambda function, so we can benefit from the fast connectivity between AWS Lambda and Amazon S3. We retrieve the relevant Source URLs and return them to our Notebook. We limit this to a 1000 links for this workshop.
```python
s3_object = s3.get_object(Bucket='gdelt-open-data', Key='events/' + file)
f = StringIO.StringIO(s3_object['Body'].read().decode('utf-8','replace').encode('ascii','replace'))
...
items = csv.DictReader(f, fieldnames, delimiter='\t')
...
for i, item in enumerate(items):
            links.append(item['SOURCEURL'])
...
return links_without_duplicates[:1000]
```




In [ ]:
futures = wrenexec.map(GDELT_scrape.get_urls_from_gdelt_data, ['20171121.export.csv'], exclude_modules=[])
links = pywren.get_all_results(futures)

Let's have a look at some of the collected links:

In [ ]:
display(links[0][:10])
display(HTML('Total links: <b>' + str(len(links[0])) + '</b>'))

Let's have a look at what some of those links look like:

In [ ]:
display(IFrame(links[0][6],1000,500))
display(IFrame(links[0][4],1000,500))

Great, now let's web scrape all those links and perform a sentiment analysis over it using NLTK and [VADER](http://www.nltk.org/_modules/nltk/sentiment/vader.html). Let's send all the links to Pywren. Here's the relevant code snippet that we will send to Pywren, explore [GDELT_scrape.py](/edit/Lab-3-Scrape-Sentiment/GDELT_scrape.py) for more details.

```python
html = urllib.urlopen(link).read()
soup = BeautifulSoup(html, "html.parser")
text = soup.get_text()
...
dynamo_tbl = boto3.resource('dynamodb', 'us-west-2').Table('pywren-workshop-gdelt-table')
sentiment = SentimentIntensityAnalyzer().polarity_scores(text)
record = {}
record['link'] = link
record['sentiment'] = str(sentiment['compound'])
words = []
for word, frequency in get_frequent_words(text):
    words.append(word + ':' + str(frequency))
record['words'] = words
response = dynamo_tbl.put_item(Item=record)
```

In [ ]:
# to optimize our function we divide all our links into 50 parts, so we send multiple links to the same Lambda function
futures = wrenexec.map(GDELT_scrape.news_analyzer, split_list(links[0], wanted_parts=50))
pywren.get_all_results(futures)

You can see the result slowly landing in the DynamoDB table - have a look at the [DynamoDB Console](https://us-west-2.console.aws.amazon.com/dynamodb/home?region=us-west-2#tables:selected=pywren-workshop-gdelt-table) to see progress. Now let's load the DynamoDB table data into our local Jupyter notebook:

In [ ]:
table = boto3.resource('dynamodb', 'us-west-2').Table('pywren-workshop-gdelt-table')
db_table = table.scan()

Let's try to understand the sentiments across all the articles that we fetched. Let's plot the overall sentiment histogram across all articles:

In [ ]:
import matplotlib.pyplot as plt
sentiments = []
for item in db_table['Items']:
    sentiments.append(float(item['sentiment']))

plt.hist(sentiments)
plt.title("Sentiment Histogram")
plt.xlabel("Sentiment")
plt.ylabel("Frequency")
plt.show()

Unsurprisingly most news articles are generally either more negative or positive with very few neutral articles. Next let's try to build a graph around frequently mentioned words. This will be across all articles:

In [ ]:
all_words = {};
for item in db_table['Items']:
    for word in item['words']:
        try:
            if word.split(':')[0] in all_words:
                all_words[word.split(':')[0]] = all_words[word.split(':')[0]] + int(word.split(':')[1])
            else:
                all_words[word.split(':')[0]] = int(word.split(':')[1])
        except ValueError:
            pass

top20 = dict(sorted(all_words.iteritems(), key=lambda (k, v): (-v, k))[:20])

x = top20.keys()
frequency = top20.values()
x_pos = [i for i, _ in enumerate(x)]
plt.figure(figsize=(10, 5))
plt.barh(x_pos, frequency, color='green')
plt.ylabel("Words used")
plt.xlabel("Frequency")
plt.title("Word frequency across articles")
plt.yticks(x_pos, x)
plt.show()

Let's have some more fun with frequently used words. How about creating a [Tag Cloud](https://en.wikipedia.org/wiki/Tag_cloud) to identify frequently mentioned words in an article? Let's try this first locally with one article:

In [ ]:
from wordcloud import WordCloud
link = db_table['Items'][0]['link']
wordcloud = WordCloud(width = 500, height = 300).generate(GDELT_scrape.scrape_content(link))
cloud = wordcloud.to_array()
plt.figure(figsize=(10, 5), dpi=100)
plt.imshow(cloud)
plt.show()

Great, how about creating such a tag cloud across all the articles? That's a perfect example for PyWren where we have a parallel mapping exercise and we can store the results straight in Amazon S3.

To accomplish this we need to change our PyWren AWS Lambda function to include the necessary libraries such as wordcloud and matplotlib. Since some of those libraries are compiled C code, PyWren will not be able to pickle it up and send it to the Lambda function. Hence we will update the entire PyWren function to include the necessary binaries that have been compiled on an Amazon EC2 instance with Amazon Linux. We pre-packaged this and made it available via [https://s3-us-west-2.amazonaws.com/pywren-workshop/wordcloud.zip](https://s3-us-west-2.amazonaws.com/pywren-workshop/wordcloud.zip)

You can simple push this code to your PyWren AWS Lambda function with below command, assuming you named the PyWren worker function with the default name pywren_1 and region us-west-2:

In [ ]:
lambdaclient = boto3.client('lambda', 'us-west-2')

response = lambdaclient.update_function_code(
    FunctionName='pywren_1',
    Publish=True,
    S3Bucket='pywren-workshop',
    S3Key='wordcloud.zip'
)

Alright, now let's use PyWren to create tagclouds of all those links in parallel and store the output into Amazon S3. Here's the relevant code snippet:
```python
from wordcloud import WordCloud
s3 = boto3.resource('s3')
for link in links:
    text = scrape_content(link)
    wordcloud = WordCloud(width = 500, height = 300).generate(text)
    cloud = wordcloud.to_file('/tmp/cloud.jpg')
    s3.Object(S3BUCKET, 'tagclouds/' + hashlib.md5(link).hexdigest() + '.jpg').put(ACL='public-read',ContentType='image/jpeg',Body=open('/tmp/cloud.jpg', 'rb'))
    return 'Ok'
```

In [ ]:
import cPickle as pickle
links = []
for item in db_table['Items']:
    links.append(item['link'])
pickle.dump(links, open('links.pickle', 'wb'))    

!python wordcloud_generator.py write --bucket_name={S3BUCKET}

Now let's try to create a small website with an overview of all these tag clouds:

In [ ]:
import hashlib

output = '<!DOCTYPE html><html><body><table><tr>'

i = 1
for link in links:
    h = hashlib.md5(link).hexdigest()
    url = 'https://s3-us-west-2.amazonaws.com/'+ S3BUCKET +'/tagclouds/' + h + '.jpg'
    output += '<td><a href="' + link + '" target="_blank"><img width="300" src="' + url + '"/></a></td>'
    if i % 5 == 0:
        output += '</tr><tr>'
    i += 1

output += '</tr></table></body></html>'

s3 = boto3.resource('s3')
response = s3.Object(S3BUCKET, 'tagclouds/index.html').put(ACL='public-read',ContentType='text/html',Body=output)
url = 'https://s3-us-west-2.amazonaws.com/'+ S3BUCKET +'/tagclouds/index.html'
display(HTML('<a href="' + url + '" target="_blank">Index Page</a>'))


That's it. We now learned how to use PyWren to request information from a large amount of URL links, generate interesting insights into sentiments, find commonly use words and render tag cloud outputs.